In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [2]:



# url_template = "https://thuvienphapluat.vn/page/tim-van-ban.aspx?keyword=m%C3%A3+s%E1%BB%91&area=0&match=True&type=21&status=0&signer=0&sort=1&lan=1&scan=0&org=0&fields=4&page={}"


# def get_links_from_page(page_number):
#     url = url_template.format(page_number)
#     response = requests.get(url)
    

#     if response.status_code != 200:
#         print(f"Lỗi khi tải trang {page_number}")
#         return []


#     soup = BeautifulSoup(response.text, 'html.parser')


#     divs = soup.find_all('div', class_=re.compile(r'content-[01]'))


#     links = [div.find('a')['href'] for div in divs if div.find('a')]
    
#     return links


# all_links = []
# for page in range(1, 401):
#     print(f"Đang tải trang {page}")
#     links = get_links_from_page(page)
#     all_links.extend(links)

# print(f"Tổng số link thu thập được: {len(all_links)}")


In [3]:
# with open('links.txt', 'w', encoding='utf-8') as f:
#     for link in all_links:
#         f.write(link + '\n')

# print("Ghi thành công các link vào file 'links.txt'")

In [39]:
links_path = os.path.join(current_directory, 'link_loi.txt')

with open(links_path, 'r', encoding='utf-8') as file:
    all_links = file.readlines()

In [40]:
len(all_links)

334

In [9]:
result_path = 'C:\\Users\\nguye\\Desktop\\results.xlsx'
workbook = load_workbook(result_path)
maso_sheet = workbook['Sheet1']
phanloai_sheet = workbook['Sheet2']

In [5]:

def get_ma_so_link_info(link):
    print(link)
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                # print(số)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'Số'.")
    except AttributeError as e:
            print(f"Lỗi: {e}")
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text:
                ngày = normalize_date(text)
                # print(ngày)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
    except AttributeError as e:
        print(f"Lỗi: {e}")
    info_list = [link.strip(),số,ngày]

    # lấy part 1
    part1 = tables[1].find_all('p')
    part1_part_1  = next((i for i, item in enumerate(part1) if "1. Hàng hóa " in item.get_text(strip=True)), -1)
    part1_part_1_text  = part1[part1_part_1].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_2 = next((i for i, item in enumerate(part1) if "Tên thương mại" in item.get_text(strip=True)), -1)
    part1_part_2_text  = part1[part1_part_2].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_3 = next((i for i, item in enumerate(part1) if "Tên gọi theo cấu tạo" in item.get_text(strip=True)), -1)
    part1_part_3_text  = part1[part1_part_3].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_4 = next((i for i, item in enumerate(part1) if "mã hiệu" in item.get_text(strip=True)), -1)
    part1_part_4_text  = part1[part1_part_4].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_5 = next((i for i, item in enumerate(part1) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
    part1_part_5_text  = part1[part1_part_5].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    info_list.append(part1_part_2_text)
    info_list.append(part1_part_3_text)
    info_list.append(part1_part_4_text)
    info_list.append(part1_part_5_text)

    # allpart
    allpart = tables[1].find_all('p')

    # lấy part 3
    pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
    for p_tag in allpart:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part3 = matches[-1]
    # lấy part 2
    try :
        part2_start = next((i for i, item in enumerate(allpart) if "2. Tóm tắt" in item.get_text(strip=True)), -1)-1
        # print(part2_start)
    except:
        part2_start = next((i for i, item in enumerate(allpart) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
        # print(part2_start)
    part2_end = next((i for i, item in enumerate(allpart) if "3. Kết quả" in item.get_text(strip=True)), -1)
    part2_list = []
    for i in allpart[part2_start+1:part2_end]:
        part2_list.append(i.get_text(strip=True))
    part2_list
    part2_text = '\n'.join(part2_list)

    info_list.append(part2_text)
    info_list.append(part3)
    print(info_list)
    return info_list

In [10]:
def write_data_to_excel(sheets_name,info_list):   
# Ghi dữ liệu vào dòng mới
    row = sheets_name.max_row + 1
    for col, value in enumerate(info_list, start=1):
        sheets_name.cell(row=row, column=col, value=value)

    # Lưu workbook
    workbook.save(result_path)

In [11]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [12]:
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        part_list.append(i.get_text(strip=True).replace('\r', '').replace('\n', '').strip())
        part_text = '\n'.join(part_list)
    return part_text
        

In [41]:
def get_phan_loai_link_info(link):
    print(link)
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                # print(số)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'Số'.")
    except AttributeError as e:
            print(f"Lỗi: {e}")
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text or 'Ngày' in text:
                ngày = normalize_date(text)
                # print(ngày)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
    except AttributeError as e:
        print(f"Lỗi: {e}")
    info_list = [link.strip(),số,ngày]

    # lấy part 1
    part1 = tables[1].find_all('p')       
    part1_1_pattern = re.compile(r"1\.\s*[Tt]ên\s*[Hh]àng")
    for p_tag in part1:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = part1_1_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part1_1_index = part1.index(p_tag)
    part1_2_pattern = re.compile(r"2\.\s*Đơn\s*vị")
    for p_tag in part1:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = part1_2_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part1_2_index = part1.index(p_tag)
    part1_3_pattern = re.compile(r"3\.\s*Số")
    part1_3_pattern_2 = re.compile(r"3\.\s*Tờ\s*khai")
    for p_tag in part1:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = part1_3_pattern.findall(text) or part1_3_pattern_2.findall(text) # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part1_3_index = part1.index(p_tag)

    part1_4_pattern = re.compile(r"4\.\s*Tóm\s*tắt")
    for p_tag in part1:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = part1_4_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part1_4_index = part1.index(p_tag)

    part1_5_pattern = re.compile(r"5\.\s*Kết\s*quả")
    for p_tag in part1:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = part1_5_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part1_5_index = part1.index(p_tag)

    part_1_text = part_text(part1,part1_1_index,part1_2_index)
    part_2_text = part_text(part1,part1_2_index,part1_3_index)
    part_3_text = part_text(part1,part1_3_index,part1_4_index)
    part_4_text = part_text(part1,part1_4_index,part1_5_index)

    pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
    for p_tag in part1[part1_5_index:]:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part_5_text = matches[-1]

    info_list.append(part_1_text)
    info_list.append(part_2_text)
    info_list.append(part_3_text)
    info_list.append(part_4_text)
    info_list.append(part_5_text)
    print(info_list)
    return info_list



In [ ]:
# all_maso_info_list = []
# for link in all_links:
#     if 'truoc-ma-so' in link:
#         try:
#             info_list = get_ma_so_link_info(link)
#             all_maso_info_list.append(info_list)
#             write_data_to_excel(maso_sheet,info_list)
#         except:
#             print(link + "lỗi")
#             info_list = [link,"lỗi"]
#             all_maso_info_list.append([link,"lỗi"])
#             write_data_to_excel(phanloai_sheet,info_list)

In [38]:
get_phan_loai_link_info('https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2915-TB-TCHQ-2018-ket-qua-phan-loai-hang-hoa-xuat-nhap-khau-vai-so-ni-393611.aspx')

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2915-TB-TCHQ-2018-ket-qua-phan-loai-hang-hoa-xuat-nhap-khau-vai-so-ni-393611.aspx
['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2915-TB-TCHQ-2018-ket-qua-phan-loai-hang-hoa-xuat-nhap-khau-vai-so-ni-393611.aspx', '2915/TB-TCHQ', '25-05-2018', '1. Tên hàng theo khai báo: Vải sơ-ni (Sofa  Fabric) khổ 1.5m, làm từ 100% polyester, loại dệt thoi, định lượng 286g/m2  hoa văn các loại, dùng trong sản xuất ghế sofa, hàng mới 100%. (Mục 1 Phụ lục  Tờ khai)', '2. Đơn vị nhập khẩu: Công ty TNHH SHAND  Việt Nam - Địa chỉ: Số 9 khu bưu điện Bình Phú, xã Bình Phú, huyện Thạch  Thất, Hà Nội. MST: 0107715811.', '3. Tờ khai số:10181879423/A11 ngày  16/01/2018 đăng ký tại Chi cục HQ CK Cảng Hải Phòng KV3 - Cục Hải quan Tp.  Hải Phòng.', '4. Tóm tắt mô tả và đặc tính hàng hóa:Sản  phẩm dệt, dạng cuộn, khổ 1,5m, trọng lượng 320,2g/m2, cấu tạo bởi:  vải dệt thoi, dệt từ các sợi đơn có màu khác nhau, thành phần 100% filament  polyest

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2915-TB-TCHQ-2018-ket-qua-phan-loai-hang-hoa-xuat-nhap-khau-vai-so-ni-393611.aspx',
 '2915/TB-TCHQ',
 '25-05-2018',
 '1. Tên hàng theo khai báo: Vải sơ-ni (Sofa  Fabric) khổ 1.5m, làm từ 100% polyester, loại dệt thoi, định lượng 286g/m2  hoa văn các loại, dùng trong sản xuất ghế sofa, hàng mới 100%. (Mục 1 Phụ lục  Tờ khai)',
 '2. Đơn vị nhập khẩu: Công ty TNHH SHAND  Việt Nam - Địa chỉ: Số 9 khu bưu điện Bình Phú, xã Bình Phú, huyện Thạch  Thất, Hà Nội. MST: 0107715811.',
 '3. Tờ khai số:10181879423/A11 ngày  16/01/2018 đăng ký tại Chi cục HQ CK Cảng Hải Phòng KV3 - Cục Hải quan Tp.  Hải Phòng.',
 '4. Tóm tắt mô tả và đặc tính hàng hóa:Sản  phẩm dệt, dạng cuộn, khổ 1,5m, trọng lượng 320,2g/m2, cấu tạo bởi:  vải dệt thoi, dệt từ các sợi đơn có màu khác nhau, thành phần 100% filament  polyester không dún, độ bền tương đối của sợi <60cN/tex, được ép dính với  lớp sản phẩm không dệt từ xơ staple polyester (chiếm 19,8% trọng lượ

In [43]:
# all_phanloai_info_list=[]
for link in all_links:
    if 'phan-loai' in link:
        try:
            info_list = get_phan_loai_link_info(link)
            all_phanloai_info_list.append(info_list)
            write_data_to_excel(phanloai_sheet,info_list)
        except:
            print(link + "lỗi")
            info_list = [link,"lỗi"]
            all_phanloai_info_list.append(info_list)
            write_data_to_excel(phanloai_sheet,info_list)

"https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-91-TB-TCHQ-phan-loai-phu-gia-dung-trong-xay-dung-dang-ma-tit-xuat-nhap-khau-2016-317340.aspx

"https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-91-TB-TCHQ-phan-loai-phu-gia-dung-trong-xay-dung-dang-ma-tit-xuat-nhap-khau-2016-317340.aspx
lỗi
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3073-TB-TCHQ-2019-ket-qua-phan-loai-doi-voi-Vai-boc-nem-ghe-ma-JSY-V0137-518072.aspx

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3073-TB-TCHQ-2019-ket-qua-phan-loai-doi-voi-Vai-boc-nem-ghe-ma-JSY-V0137-518072.aspx', '3073/TB-TCHQ', '20-05-2019', '1. Tên hàng theo khai báo:\nVải bọc nệm ghế mã JSY-V0137 (dệt  thoi từ bông, khổ1,4m) dùng sản xuất nệm ghế, mới 100%.  (Mục 2 Phụ lục Tờ khai)', '2. Đơn vị xuấtnhập khẩu:Công ty TNHH Thương mại Dịch vụ  Thành Lộc Phát. Địa chỉ: Thửa đất số 76, tờ bản đồ số 46, ấp Kiến An, xã An  Lập, huyện Dầu Tiếng, tỉnh Bình Dương. MST: 3702721375.', '3. Tờ khai số:1024260

In [44]:
len(all_phanloai_info_list)

6046

In [35]:
# with open('output.txt', 'w') as f:
#     # Duyệt qua từng list con và ghi từng dòng vào file
#     for row in all_phanloai_info_list:
#         # Chuyển list con thành chuỗi với các phần tử ngăn cách bởi dấu phẩy
#         row_str = ', '.join(map(str, row))
#         # Ghi vào file và xuống dòng
#         f.write(row_str + '\n')

In [45]:
# wb2 = Workbook()
# ws2 = wb2.active  # Lấy sheet mặc định

# # Ghi từng dòng của danh sách vào các ô Excel
# for row in all_phanloai_info_list:
#     ws2.append(row)  # Append từng list con vào sheet

# # Lưu workbook vào file Excel
# wb2.save("output.xlsx")

# print("Danh sách hai chiều đã được ghi vào file 'output.xlsx'.")

Danh sách hai chiều đã được ghi vào file 'output.xlsx'.
